In [10]:
import pandas as pd
import tensorflow as tf
from transformers import TFRobertaForSequenceClassification, RobertaTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Load the dataset
data = pd.read_csv("train.csv") 

# Preprocess the dataset
df = pd.DataFrame(data)

# Split into training and validation sets (80% for training, 20% for validation)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [11]:
# Check for available devices
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is NOT available")


GPU is available


In [12]:
# Load the RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Tokenize the training data
train_encodings = tokenizer(
    train_df['Claim'].tolist(),
    train_df['Evidence'].tolist(),
    padding=True,
    truncation=True,
    return_tensors='tf'
)

# Tokenize the validation data
val_encodings = tokenizer(
    val_df['Claim'].tolist(),
    val_df['Evidence'].tolist(),
    padding=True,
    truncation=True,
    return_tensors='tf'
)

# Create tf.data.Dataset for training and validation
train_dataset = tf.data.Dataset.from_tensor_slices((
    train_encodings['input_ids'],
    train_df['label'].tolist()
)).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    val_encodings['input_ids'],
    val_df['label'].tolist()
)).batch(16)

In [15]:
# Load the model with binary classification
model = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=1)

# Compile the model with binary cross-entropy and an Adam optimizer
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),  
    metrics=['accuracy']
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [16]:
model.fit(train_dataset, validation_data=val_dataset, epochs=3)# Fine-Tune the Model

Epoch 1/3
1186/1186 [==============================] - 1766s 1s/step - loss: 0.5351 - accuracy: 0.7413 - val_loss: 0.3978 - val_accuracy: 0.8235
Epoch 2/3
1186/1186 [==============================] - 1667s 1s/step - loss: 0.3238 - accuracy: 0.8479 - val_loss: 0.3368 - val_accuracy: 0.8606
Epoch 3/3
1186/1186 [==============================] - 1669s 1s/step - loss: 0.2465 - accuracy: 0.8886 - val_loss: 0.3312 - val_accuracy: 0.8599


In [17]:
# Load the dev dataset for evaluation
dev_data = pd.read_csv("dev.csv")

df = pd.DataFrame(data) # Preprocess the dataset


In [18]:
#Tokenize the dev dataset inputs
dev_encodings = tokenizer(
    df['Claim'].tolist(),
    df['Evidence'].tolist(),
    padding=True,
    truncation=True,
    return_tensors='pt'
)

# Create a TensorFlow dataset for dev
dev_dataset = tf.data.Dataset.from_tensor_slices((
    dev_encodings['input_ids'],  # Tokenized input data
    df['label'].tolist()  # Corresponding labels
)).batch(16)

# Evaluate the model to get test loss and accuracy
test_loss, test_accuracy = model.evaluate(dev_dataset)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

1482/1482 [==============================] - 632s 418ms/step - loss: 0.2070 - accuracy: 0.9219
Test Loss: 0.2070
Test Accuracy: 0.9219


In [20]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Predict the logits from the model
predicted_logits = model.predict(dev_dataset).logits  # Get logits

# Convert logits to probabilities
predicted_probabilities = tf.nn.sigmoid(predicted_logits)

# Convert probabilities to binary predictions (threshold at 0.5)
y_pred = (predicted_probabilities > 0.5).numpy().astype(int)  # Convert to integers

y_test = df['label'].tolist() # Get the true labels

precision = precision_score(y_test, y_pred) # Calculate precision
recall = recall_score(y_test, y_pred) # Calculate recall
f1 = f1_score(y_test, y_pred) # Calculate F1-score

print(f'Precision: {precision}, Recall: {recall}, F1-score: {f1}')


1482/1482 [==============================] - 611s 412ms/step
Precision: 0.7821616758771366, Recall: 0.9364517618095092, F1-score: 0.8523809523809524


In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

import shutil

# Path to the desired folder in Google Drive
drive_folder_path = "/content/drive/MyDrive/my_models"

# Create the folder if it doesn't exist
if not os.path.exists(drive_folder_path):
    os.makedirs(drive_folder_path)

# Save the model to the folder
model.save(f"{drive_folder_path}/roberta_saved_model", save_format="tf")  # Save the model in the SavedModel format
